Part of DEAP Library

Goal is to get exactly $15.05 worth of appetizers, as fast as possible

In [118]:
import random

In [119]:
from operator import attrgetter
from collections import Counter

try:
    del Counter.__reduce__
except: pass

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

## Create Item Dictionary

In [120]:
ITEMS = {'French Fries': (2.75, 5),
        'Hot Wings' : (3.55, 7),
        'Mixed Fruit' : (2.15, 2),
        'Mozzarella Sticks' : (4.2, 4),
        'Sampler Plate' : (5.8, 10),
        'Side Salad' : (3.35, 3)}
ITEMS_NAME = list(ITEMS.keys())

## Fitness Function

Our fitness function will have 3 values:

1) Cost difference from target price

2) Time to eat

3) Amount of food

-1 is trying to minimize value, 1 is trying to maximize value

In [121]:
creator.create('FitnessMulti', base.Fitness, weights=(1.0,-1.0,1.0))
creator.create('Individual', Counter, fitness = creator.FitnessMulti)

#Start with 3 items
IND_INIT_SIZE = 3

toolbox = base.Toolbox()
toolbox.register('attr_item', random.choice, ITEMS_NAME)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

In [122]:
toolbox.population(n=5)

[Individual({'Hot Wings': 1, 'Side Salad': 1, 'French Fries': 1}),
 Individual({'Side Salad': 1, 'Sampler Plate': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mixed Fruit': 1, 'Side Salad': 1, 'Mozzarella Sticks': 1}),
 Individual({'Mozzarella Sticks': 1, 'Sampler Plate': 1, 'Hot Wings': 1}),
 Individual({'French Fries': 1, 'Sampler Plate': 1, 'Hot Wings': 1})]

## Evaluate Fitness and Return Error

In [123]:
def evalXKCD(individual, target_price):
    price = 0.0
    times = [0]
    food = 0
    for item, number in individual.items():
        if number > 0:
            price += ITEMS[item][0] * number
            times.append(ITEMS[item][1])
            food += number
    return (price - target_price), max(times), food

In [124]:
evalXKCD(creator.Individual({'French Fries' : 2, 'Hot Wings' : 0, 'Mixed Fruit' : 0, 
                            'Mozzarella Sticks' : 0, 'Sampler Plate' : 0, 'Side Salad' : 2}), 15.05)

(-2.8500000000000014, 5, 4)

## Swap The Number of Randomly-chosen Items Between 2 Individuals

In [125]:
def cxCounter(ind1, ind2, indpb):
    for key in ITEMS.keys():
        if random.random() < indpb:
            ind1[key], ind2[key] = ind2[key], ind1[key]
    return ind1, ind2

In [126]:
cxCounter(creator.Individual({'French Fries' : 4, 'Hot Wings' : 0, 'Mixed Fruit' : 0, 
                            'Mozzarella Sticks' : 0, 'Sampler Plate' : 0, 'Side Salad' : 3}),
         creator.Individual({'French Fries' : 0, 'Hot Wings' : 1, 'Mixed Fruit' : 0, 
                            'Mozzarella Sticks' : 0, 'Sampler Plate' : 1, 'Side Salad' : 2}),
         0.5)

(Individual({'French Fries': 4,
             'Hot Wings': 1,
             'Mixed Fruit': 0,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 1,
             'Side Salad': 2}),
 Individual({'French Fries': 0,
             'Hot Wings': 0,
             'Mixed Fruit': 0,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 0,
             'Side Salad': 3}))

## Add or Remove items from an individual

In [127]:
def mutCounter(individual):
    if random.random() > 0.5:
        individual.update([random.choice(ITEMS_NAME)])
    else:
        val = random.choice(ITEMS_NAME)
        individual.subtract([val])
        if individual[val] < 0:
            del individual[val]
    return individual,

In [128]:
mutCounter(creator.Individual({'French Fries' : 4, 'Hot Wings' : 0, 'Mixed Fruit' : 0, 
                            'Mozzarella Sticks' : 0, 'Sampler Plate' : 0, 'Side Salad' : 3}))

(Individual({'French Fries': 4,
             'Hot Wings': 0,
             'Mixed Fruit': 0,
             'Mozzarella Sticks': 0,
             'Sampler Plate': 0,
             'Side Salad': 4}),)

In [129]:
import sys

toolbox.register('evaluate', evalXKCD, target_price=15.05)

#Severely penalize targets over budget
toolbox.decorate('evaluate', tools.DeltaPenalty(lambda ind: evalXKCD(ind, 15.05)[0] <= 0,
                                               (-sys.float_info.max,
                                               sys.float_info.max,
                                               -sys.float_info.max)))

toolbox.register('mate', cxCounter, indpb = 0.5)
toolbox.register('mutate', mutCounter)
toolbox.register('select', tools.selNSGA2)

## Simulation Parameters

In [130]:
# Number of generations = 100
NGEN = 100;
# Number of selected individuals
MU = 50;
# Number of children to produce
LAMBDA = 20;
# Probabilty offspring is produced by crossover
CXPB = 0.3;
# Probability offspring is produced by mutation
MUTPB = 0.3;

# Start population at 50
pop = toolbox.population(n=MU);

## Keep Track of the 'Hall of Fame' Individual

Hall of Fame contains the best individual that ever lived in the population during evolution

In [131]:
hof = tools.ParetoFront()

price_stats = tools.Statistics(key = lambda ind: ind.fitness.values[0])
time_stats = tools.Statistics(key = lambda ind: ind.fitness.values[1])
food_stats = tools.Statistics(key = lambda ind: ind.fitness.values[2])

stats = tools.MultiStatistics(price = price_stats, time = time_stats, food = food_stats)
stats.register('avg', numpy.mean, axis = 0)

algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof)

   	      	         food         	        price         	         time         
   	      	----------------------	----------------------	----------------------
gen	nevals	avg	gen	nevals	avg   	gen	nevals	avg 	gen	nevals
0  	50    	3  	0  	50    	-4.051	0  	50    	7.96	0  	50    
1  	11    	3.04	1  	11    	-3.789	1  	11    	7.68	1  	11    
2  	9     	3.14	2  	9     	-3.505	2  	9     	7.44	2  	9     
3  	11    	3.14	3  	11    	-3.753	3  	11    	6.86	3  	11    
4  	8     	3.26	4  	8     	-3.611	4  	8     	6.44	4  	8     
5  	12    	3.3 	5  	12    	-4.117	5  	12    	5.68	5  	12    
6  	12    	3.48	6  	12    	-3.565	6  	12    	5.84	6  	12    
7  	10    	3.56	7  	10    	-3.41 	7  	10    	5.5 	7  	10    
8  	13    	3.6 	8  	13    	-3.513	8  	13    	5.08	8  	13    
9  	11    	3.6 	9  	11    	-3.397	9  	11    	4.86	9  	11    
10 	12    	3.54	10 	12    	-3.969	10 	12    	4.32	10 	12    
11 	15    	3.64	11 	15    	-3.546	11 	15    	4.4 	11 	15    
12 	14    	3.8 	12 	14    	-3.292	12 	14    	4.36

([Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2,
              'Sampler Plate': 0}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2,
              'Sampler Plate': 0}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2,
              'Sampler Plate': 0}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
              'Mozzarella Sticks': 0,
              'Mixed Fruit': 2,
              'Sampler Plate': 0}),
  Individual({'Side Salad': 1,
              'Hot Wings': 2,
     

In [132]:
print('Best:', hof[0], hof[0].fitness.values)

Best: Individual({'Hot Wings': 2, 'Mixed Fruit': 2, 'Side Salad': 1, 'Mozzarella Sticks': 0, 'French Fries': 0}) (-0.3000000000000007, 7.0, 5.0)
